In [12]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import numpy as np
import urllib.parse
import re

In [16]:
class Vacancies:
    """vacancies in Moscow on hh.ru"""
    
    def __init__(self):
        pass
    
    def find(self, yourtext, n_pages, site):
        header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
          'Accept':'*/*'}
        vacanсies = []
        
        if site == 'hh':
            main_link = 'https://hh.ru/search/vacancy'
            
            for i in range(n_pages):
                params = {'area': 1, 'st': 'searchVacancy', 'text': yourtext, 'page': i}
                response = requests.get(main_link, params=params, headers = header)
                soup = bs(response.text,'lxml')

                vacanсies_block = soup.find('div',{'class':'vacancy-serp'})
                vacanсies_list = vacanсies_block('div',{'class':'vacancy-serp-item'})

                for element in vacanсies_list:
                    element_data = {}
                    element_link = None
                    element_name = None
                    element_employer_name = None
                    element_address = None
                    salary_text = None
                    salary_min = None
                    salary_max = None
                    salary_crrency = None

                    element_link = element.find(attrs={'data-qa': 'vacancy-serp__vacancy-title'}, href=True)['href'].split('?')[0]
                    element_name = element.find(attrs={'data-qa': 'vacancy-serp__vacancy-title'}).getText()
                    element_employer_name = element.find(attrs={'data-qa': 'vacancy-serp__vacancy-employer'}).getText()
                    element_address = element.find(attrs={'data-qa': 'vacancy-serp__vacancy-address'}).getText()
                    salary_text = element.find(attrs={'data-qa': 'vacancy-serp__vacancy-compensation'})

                    if salary_text:
                        salary_text = salary_text.getText().replace(u'\xa0', u'')
                        salaries =  re.findall('\d+', salary_text)
                        text = re.findall('[a-zA-Zа-яА-Я]+', salary_text)
                        if 'до' in text:
                            salary_max = salaries[0]
                            salary_crrency = text[1]
                        elif 'от' in text:
                            salary_min = salaries[0]
                            salary_crrency = text[1]
                        elif len(salaries) == 2:
                            salary_min = salaries[0]
                            salary_max = salaries[1]
                            salary_crrency = text[0]

                    element_data['name'] = element_name
                    element_data['link'] = element_link
                    element_data['salary_min'] = salary_min
                    element_data['salary_max'] = salary_max
                    element_data['salary_currency'] = salary_crrency
                    element_data['employer'] = element_employer_name
                    element_data['address'] = element_address
                    element_data['site'] = 'hh.ru'

                    vacanсies.append(element_data)

                next_button = soup.find(attrs={'data-qa': 'pager-next'}, href=True)
                if not next_button:
                    break  
                    
        return vacanсies
    
    def get_table(self, vacanсies):
        df = pd.DataFrame.from_dict(vacanсies)
        return df

In [17]:
vacancies = Vacancies()
yourtext = 'python разработчик'
n_pages = 4
site = 'hh'

In [18]:
my_list = vacancies.find(yourtext, n_pages, site)
df = vacancies.get_table(my_list)

In [19]:
df.head()

,name,link,salary_min,salary_max,salary_currency,employer,address,site
0,Back-end Developer (Node/Java/Python),https://hh.ru/vacancy/46222999,None,None,None,Сбер. IT,Москва,hh.ru
1,Python разработчик,https://hh.ru/vacancy/46314277,250,None,руб,ООО МОНСТР,"Москва, Курская",hh.ru
2,Rust разработчик,https://hh.ru/vacancy/45470238,None,None,None,Group-IB,Москва,hh.ru
3,DevOps Инженер (Python),https://hh.ru/vacancy/46408662,None,None,None,Сбер. IT,"Москва, Кутузовская и еще 1",hh.ru
4,Python developer / data engineer (Jun->Mid),https://hh.ru/vacancy/46392030,None,None,None,Wunder Fund,Москва,hh.ru


In [20]:
df.tail()

,name,link,salary_min,salary_max,salary_currency,employer,address,site
75,Python Developer,https://hh.ru/vacancy/42358806,None,None,None,Онлайн-школа Тетрика,"Москва, Курская",hh.ru
76,Разработчик Python,https://hh.ru/vacancy/44010522,None,None,None,Forecsys,Москва,hh.ru
77,Senior Python Developer,https://hh.ru/vacancy/45835639,None,3,EUR,ООО Globus.ai,Москва,hh.ru
78,Go/Python разработчик (Middle),https://hh.ru/vacancy/45842425,None,None,None,ANCOR,Москва,hh.ru
79,Программист Python,https://hh.ru/vacancy/43941823,None,None,None,ООО Лендсбэй,Москва,hh.ru


In [21]:
len(my_list)

80